In [38]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data_path = "../01_Data/"
file_name_class = "psp-data_class.csv"
file_name_class_feat_sel = "psp-data_class-feat-sel.csv"
file_name_class_pcb = "psp-data_class-pcb.csv"
# file_path_class = data_path + file_name_class
# file_path_class = data_path + file_name_class_feat_sel
file_path_class = data_path + file_name_class_pcb

In [39]:
df = pd.read_csv(file_path_class, index_col=0)
df

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,success
0,-0.857406,0.443517,-0.613989,-0.016817,-0.180887,-0.492122,-0.045587,0
1,-0.857406,0.443517,-0.613989,-0.016817,-0.180887,-0.492122,-0.045587,1
2,-0.688943,0.454114,0.865572,-0.169420,0.690152,-0.494667,-0.049661,0
3,-0.688943,0.454114,0.865572,-0.169420,0.690152,-0.494667,-0.049661,1
4,-0.489985,-0.881992,0.719552,-0.574483,-0.303899,-0.496782,-0.101599,0
...,...,...,...,...,...,...,...,...
50324,-0.849292,0.442867,-0.613758,-0.017870,-0.177859,0.512640,-0.028067,0
50325,0.436694,0.628494,-0.059611,-0.071829,-0.231405,0.496592,-0.020960,0
50326,0.436694,0.628494,-0.059611,-0.071829,-0.231405,0.496592,-0.020960,0
50327,0.436694,0.628494,-0.059611,-0.071829,-0.231405,0.496592,-0.020960,0


In [ ]:
h2o.init()

In [41]:
psp_df = h2o.import_file(file_path_class)

psp_train, psp_test, psp_valid = psp_df.split_frame(ratios=[.7, .15])

y = "success"
x = psp_df.columns
x.remove(y)
x.remove("C1")
# x.remove("tmsp")
# # wenn "fee" drinnen bleibt, dann kann eine Vorhersage zu 100% durchgeführt werden. 
# # Die "fee" ist jedoch abhängig von "success" und "PSP"
# x.remove("fee")
# x.remove("amount_bucket")
# x.remove("purchase")
# x.remove("PSP")
# x.remove("card")
# x.remove("country")
psp_train[y] = psp_train[y].asfactor()
psp_test[y] = psp_test[y].asfactor()

aml = H2OAutoML(max_models=20, seed =10, exclude_algos=["StackedEnsemble", "DeepLearning"], verbosity="info", nfolds=0)
#aml = H2OAutoML(max_models=20, seed =10, verbosity="info", nfolds=0)

aml.train(x=x, y=y, training_frame=psp_train, validation_frame=psp_valid)


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
22:12:52.401: Project: AutoML_1_20240621_221252
22:12:52.402: Cross-validation disabled by user: no fold column nor nfolds > 1.
22:12:52.407: Setting stopping tolerance adaptively based on the training frame: 0.005335021838556467
22:12:52.407: Build control seed: 10
22:12:52.408: training frame: Frame key: AutoML_1_20240621_221252_training_py_45_sid_ac14    cols: 9    rows: 35134  chunks: 32    size: 2088030  checksum: 6333163672276517195
22:12:52.408: validation frame: Frame key: py_44_sid_ac14    cols: 9    rows: 7636  chunks: 32    size: 463699  checksum: -8334007964576113840
22:12:52.408: leaderboard frame: Frame key: py_44_sid_ac14    cols: 9    rows: 7636  chunks: 32    size: 463699  checksum: -8334007964576113840
22:12:52.408: blending frame: NULL
22:12:52.408: response column: success
22:12:52.408: fold column: null
22:12:52.408: weights column: null
22:12:52.418: A

Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_2_AutoML_1_20240621_221252


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    40                 40                          44708                  7            7            7             47            114           84.4

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.14704395206586718
RMSE: 0.38346310391727023
LogLoss: 0.46221444982595994
Mean Per-Class Error: 0.3553839074243375
AUC: 0.7001798639141134
AUCPR: 0.39601847109685234
Gini: 0.4003597278282267

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.20483308263654265
       0      1      Error    Rate
-----  -----  -----  -------  -----------------
0      18779  9245   0.3299   (9245.0/28024.0)
1      2708   4402   0.3809   (2708.0/7110.0)
Total  21487  13647  0.3402   (11953.0/35134.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.204833     0.424146  249
max f2                       0.150434     0.58489   315
max f0point5                 0.299882     0.394337  165
max accuracy                 0.411635     0.805288  104
max precision                0.84444      1         0
max recall                   0.066834     1         395
max specificity              0.84444      1         0
max absolute_mcc             0.213963     0.240074  240
max min_per_class_accuracy   0.200642     0.642757  254
max mean_per_class_accuracy  0.204833     0.644616  249
max tns                      0.84444      28024     0
max fns                      0.84444      7108      0
max fps                      0.0508578    28024     399
max tps                      0.066834     7110      395
max tnr                      0.84444      1         0
max fnr                      0.84444      0.999719  0
max fpr                      0.0508578    1         399
max tpr                      0.066834     1         395

Gains/Lift Table: Avg response rate: 20,24 %, avg score: 20,23 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100472                   0.554786           3.79361   3.79361            0.767705         0.641728   0.767705                    0.641728            0.0381153       0.0381153                  279.361   279.361            0.0351893
2        0.0200376                   0.484803           2.95645   3.37622            0.598291         0.513423   0.683239                    0.577758            0.0295359       0.0676512                  195.645   237.622            0.0596937
3        0.0301417                   0.432675           2.51946   3.08902            0.509859         0.457424   0.625118                    0.537419            0.0254571       0.0931083                  151.946   208.902            0.0789419
4        0.0402459                   0.400576           2.44987   2.92855            0.495775         0.414039   0.592645                    0.506443            0.0247539       0.117862                   144.987   192.855            0.0973084
5        0.0501509                   0.383606           2.41395   2.82691            0.488506         0.391533   0.572077     

In [42]:
lb = aml.leaderboard

In [43]:
print(lb)

model_id                                           auc    logloss     aucpr    mean_per_class_error      rmse       mse
GBM_2_AutoML_1_20240621_221252                0.650903   0.488534  0.33447                 0.39185   0.396379  0.157116
GBM_grid_1_AutoML_1_20240621_221252_model_11  0.650861   0.488154  0.338458                0.386505  0.396006  0.15682
GBM_1_AutoML_1_20240621_221252                0.649483   0.488081  0.337707                0.380283  0.395933  0.156763
GBM_grid_1_AutoML_1_20240621_221252_model_7   0.648658   0.488413  0.336367                0.3857    0.396048  0.156854
GBM_grid_1_AutoML_1_20240621_221252_model_12  0.647296   0.489065  0.33581                 0.390109  0.396401  0.157133
GBM_3_AutoML_1_20240621_221252                0.647066   0.489854  0.329869                0.386009  0.396988  0.157599
GBM_grid_1_AutoML_1_20240621_221252_model_5   0.644967   0.490394  0.330642                0.394519  0.397048  0.157647
GBM_5_AutoML_1_20240621_221252           

In [44]:
psp_pred = aml.leader.predict(psp_test)
psp_pred.head()

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


predict,p0,p1
0,0.876646,0.123354
1,0.805702,0.194298
1,0.789057,0.210943
0,0.851702,0.148298
0,0.886686,0.113314
0,0.886686,0.113314
1,0.396321,0.603679
0,0.871212,0.128788
0,0.884964,0.115036
0,0.884964,0.115036


In [45]:
aml.leader.model_performance(psp_test)

ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.15433288235722756
RMSE: 0.392852239852629
LogLoss: 0.48259244556886416
Mean Per-Class Error: 0.39533116976494886
AUC: 0.6405033401312521
AUCPR: 0.30579191373328274
Gini: 0.2810066802625042

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.18480797580928185
       0     1     Error    Rate
-----  ----  ----  -------  ---------------
0      3207  2832  0.469    (2832.0/6039.0)
1      489   1031  0.3217   (489.0/1520.0)
Total  3696  3863  0.4393   (3321.0/7559.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.184808     0.383058  262
max f2                       0.137567     0.564762  321
max f0point5                 0.287154     0.322432  152
max accuracy                 0.565331     0.799709  34
max precision                0.623761     0.55814   23
max recall                   0.0652815    1         396
max specificity              0.811764     0.999834  0
max absolute_mcc             0.202261     0.168147  237
max min_per_class_accuracy   0.197085     0.597368  244
max mean_per_class_accuracy  0.184808     0.604669  262
max tns                      0.811764     6038      0
max fns                      0.811764     1520      0
max fps                      0.0527595    6039      399
max tps                      0.0652815    1520      396
max tnr                      0.811764     0.999834  0
max fnr                      0.811764     1         0
max fpr                      0.0527595    1         399
max tpr                      0.0652815    1         396

Gains/Lift Table: Avg response rate: 20,11 %, avg score: 20,22 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score     cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  --------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100542                   0.557184           2.68282   2.68282            0.539474         0.641009  0.539474                    0.641009            0.0269737       0.0269737                  168.282   168.282            0.021178
2        0.0202408                   0.477331           2.06671   2.37275            0.415584         0.515314  0.477124                    0.577751            0.0210526       0.0480263                  106.671   137.275            0.0347791
3        0.0300304                   0.424023           2.2849    2.34411            0.459459         0.449237  0.471366                    0.535857            0.0223684       0.0703947                  128.49    134.411            0.0505239
4        0.0400847                   0.391367           1.43956   2.11723            0.289474         0.40502   0.425743                    0.50304             0.0144737       0.0848684                  43.956    111.723            0.0560557
5        0.0500066                   0.375096           1.65768   2.02605            0.333333         0.384897  0.407407                    0.479598            0.0164474       0.101316                   65.7675   102.605            0.0642236
6        0.100013                    0.309384           1.63136   1.82871            0.328042         0.343265  0.367725                    0.411432            0.0815789       0.182895                   63.1363   82.8705            0.103743
7        0.15002                     0.276333           1.40771   1.68837            0.283069         0.28948   0.339506                    0.370781            0.0703947       0.253289                   40.7709   68.8373            0.129262
8        0.200291             

In [ ]:
h2o.shutdown(prompt=False)